# Empirical Application: The DML Condition Number Diagnostic

**Paper Reference**: Saco (2025), "Finite-Sample Failures and Condition-Number Diagnostics in Double Machine Learning"

---

This notebook demonstrates the practical relevance of the DML condition number $\kappa_{\text{DML}}$ using the canonical LaLonde (1986) job training dataset. We show that:

1. **Well-conditioned designs** (experimental sample): $\kappa_{\text{DML}} \approx 1$–4, delivering reliable inference
2. **Ill-conditioned designs** (observational sample): $\kappa_{\text{DML}} \gg 10$, leading to inflated standard errors and unreliable CIs

The condition number is defined as:
$$\kappa_{\text{DML}} := \frac{1}{|\hat{J}_\theta|} = \frac{n}{\sum_{i=1}^n \hat{U}_i^2}$$
where $\hat{U}_i = D_i - \hat{m}(X_i)$ are the cross-fitted treatment residuals.

In [ ]:
# =============================================================================
# Empirical Application: LaLonde/NSW Data Analysis
# =============================================================================
# This notebook demonstrates the DML condition number diagnostic (κ_DML)
# using the canonical LaLonde (1986) job training dataset.
#
# Reference: Saco (2025), "Finite-Sample Failures and Condition-Number 
# Diagnostics in Double Machine Learning", The Econometrics Journal.
# =============================================================================

# Standard library
import sys
import warnings
warnings.filterwarnings('ignore')

# Scientific computing
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import seaborn as sns

# =============================================================================
# Publication-Quality Figure Settings (Econometrics Journal Style)
# =============================================================================
plt.style.use('seaborn-v0_8-whitegrid')

# LaTeX-style fonts
plt.rcParams.update({
    # Figure size and DPI
    'figure.figsize': (6.5, 4.5),
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.05,
    
    # Font settings (matching LaTeX)
    'font.family': 'serif',
    'font.serif': ['Computer Modern Roman', 'Times New Roman', 'DejaVu Serif'],
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    
    # Use LaTeX for text rendering (if available)
    'text.usetex': False,  # Set True if LaTeX is installed
    'mathtext.fontset': 'cm',
    
    # Lines and markers
    'lines.linewidth': 1.5,
    'lines.markersize': 6,
    
    # Axes
    'axes.linewidth': 0.8,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linewidth': 0.5,
    
    # Legend
    'legend.frameon': True,
    'legend.framealpha': 0.9,
    'legend.edgecolor': '0.8',
    
    # Tight layout
    'figure.constrained_layout.use': True,
})

# Color palette for academic figures
COLORS = {
    'experimental': '#2C7BB6',  # Blue
    'observational': '#D7191C',  # Red
    'LIN': '#1B9E77',           # Teal
    'LASSO': '#D95F02',         # Orange
    'RF': '#7570B3',            # Purple
    'benchmark': '#66A61E',     # Green
    'threshold': '#E7298A',     # Magenta
    'neutral': '#666666',       # Gray
}

# Ensure reproducibility
np.random.seed(42)

# Add src to path if needed
sys.path.insert(0, '..')

print("=" * 60)
print("DML Condition Number Diagnostic: Empirical Application")
print("=" * 60)
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {mpl.__version__}")

: 

In [ ]:
# Import our custom modules
from empirical.data_lalonde import (
    load_lalonde_data, 
    get_experimental_sample,
    get_observational_sample,
    get_covariate_matrix,
    summary_statistics
)

from empirical.dml_kappa import (
    PLRDoubleMLKappa,
    get_learner,
    estimate_propensity_score,
    compute_overlap_diagnostics,
    run_dml_analysis
)

from empirical.utils_tables import (
    results_to_dataframe,
    combine_results_tables,
    format_results_table,
    results_to_latex,
    plot_propensity_histogram,
    plot_overlap_comparison,
    plot_kappa_by_design,
    plot_ci_length_vs_kappa
)

print("Custom modules loaded successfully.")

---

## 1. Data Loading and Sample Construction

We load the LaLonde/NSW data and construct two samples:

1. **Experimental sample**: NSW participants + NSW control group (randomised)
2. **Observational sample**: NSW participants + CPS/PSID comparison group (non-experimental)

The experimental sample provides a benchmark where treatment is randomised, while the observational sample illustrates the challenges of conditioning on covariates.

In [ ]:
# Load LaLonde/NSW data

df = load_lalonde_data(include_psid=True, include_cps=False, verbose=False)


print("\n=== Dataset Summary ===")

print(f"Full dataset: {len(df)} observations")

print(f"Treated: {int(df['D'].sum())}, Controls: {int((df['D'] == 0).sum())}")


In [ ]:
# Construct experimental and observational samples

df_exp = get_experimental_sample(df)

df_obs = get_observational_sample(df)



print(f"Experimental sample: n = {len(df_exp)}, n_treated = {int(df_exp['D'].sum())}")

print(f"Observational sample: n = {len(df_obs)}, n_treated = {int(df_obs['D'].sum())}")


In [ ]:
# Summary statistics for key variables
covariates = ['age', 'education', 'married', 'nodegree', 're74', 're75']

print("\n=== Experimental Sample ===")
display(summary_statistics(df_exp, covariates + ['re78']))

print("\n=== Observational Sample ===")
display(summary_statistics(df_obs, covariates + ['re78']))

### Covariate Balance

Compare covariate distributions between treated and control groups in each sample.

In [ ]:
def covariate_balance_table(df, covariates, treatment_col='treat'):
    """Compute standardised mean differences for covariate balance."""
    treated = df[df[treatment_col] == 1]
    control = df[df[treatment_col] == 0]
    
    balance = []
    for cov in covariates:
        mean_t = treated[cov].mean()
        mean_c = control[cov].mean()
        std_t = treated[cov].std()
        std_c = control[cov].std()
        pooled_std = np.sqrt((std_t**2 + std_c**2) / 2)
        smd = (mean_t - mean_c) / pooled_std if pooled_std > 0 else 0
        balance.append({
            'Covariate': cov,
            'Mean (Treated)': mean_t,
            'Mean (Control)': mean_c,
            'SMD': smd
        })
    
    return pd.DataFrame(balance)

print("=== Covariate Balance: Experimental ===")
balance_exp = covariate_balance_table(df_exp, covariates)
display(balance_exp.style.format({'Mean (Treated)': '{:.2f}', 'Mean (Control)': '{:.2f}', 'SMD': '{:.3f}'}))

print("\n=== Covariate Balance: Observational ===")
balance_obs = covariate_balance_table(df_obs, covariates)
display(balance_obs.style.format({'Mean (Treated)': '{:.2f}', 'Mean (Control)': '{:.2f}', 'SMD': '{:.3f}'}))

---

## 2. Propensity Score Overlap Diagnostics

Before running DML, we examine propensity score overlap. Poor overlap is a key driver of large $\kappa_{\text{DML}}$.

In [ ]:
# Estimate propensity scores using logistic regression
from sklearn.linear_model import LogisticRegression

X_exp = get_covariate_matrix(df_exp, covariates)
D_exp = df_exp['treat'].values

X_obs = get_covariate_matrix(df_obs, covariates)
D_obs = df_obs['treat'].values

# Fit logistic regression
ps_model_exp = LogisticRegression(max_iter=1000, solver='lbfgs')
ps_model_exp.fit(X_exp, D_exp)
ps_exp = ps_model_exp.predict_proba(X_exp)[:, 1]

ps_model_obs = LogisticRegression(max_iter=1000, solver='lbfgs')
ps_model_obs.fit(X_obs, D_obs)
ps_obs = ps_model_obs.predict_proba(X_obs)[:, 1]

In [ ]:
# Overlap diagnostics

overlap_exp = compute_overlap_diagnostics(D_exp, X_exp)

overlap_obs = compute_overlap_diagnostics(D_obs, X_obs)



print("=== Overlap Diagnostics ===")

print(f"\nExperimental sample:")

for k, v in overlap_exp.items():

    # Avoid formatting arrays

    if isinstance(v, (np.ndarray, list)):

        print(f"  {k}: array with shape {np.asarray(v).shape}")

    else:

        print(f"  {k}: {v:.4f}")



print(f"\nObservational sample:")

for k, v in overlap_obs.items():

    if isinstance(v, (np.ndarray, list)):

        print(f"  {k}: array with shape {np.asarray(v).shape}")

    else:

        print(f"  {k}: {v:.4f}")


In [ ]:
# =============================================================================
# Figure 1: Propensity Score Overlap (Publication Quality)
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(7, 3.5))

# Panel (a): Experimental Sample
ax = axes[0]
ax.hist(ps_exp[D_exp == 0], bins=25, alpha=0.7, label='Control', 
        density=True, color=COLORS['experimental'], edgecolor='white', linewidth=0.5)
ax.hist(ps_exp[D_exp == 1], bins=25, alpha=0.7, label='Treated', 
        density=True, color=COLORS['observational'], edgecolor='white', linewidth=0.5)
ax.axvline(0.1, color=COLORS['neutral'], linestyle='--', linewidth=1, alpha=0.8)
ax.axvline(0.9, color=COLORS['neutral'], linestyle='--', linewidth=1, alpha=0.8)
ax.axvline(0.5, color='black', linestyle='-', linewidth=0.8, alpha=0.5)
ax.set_xlabel(r'Propensity Score $\hat{e}(X)$')
ax.set_ylabel('Density')
ax.set_title('(a) Experimental Sample', fontweight='bold')
ax.legend(loc='upper right', framealpha=0.9)
ax.set_xlim([-0.02, 1.02])
ax.set_ylim(bottom=0)

# Panel (b): Observational Sample  
ax = axes[1]
ax.hist(ps_obs[D_obs == 0], bins=25, alpha=0.7, label='Control (PSID)', 
        density=True, color=COLORS['experimental'], edgecolor='white', linewidth=0.5)
ax.hist(ps_obs[D_obs == 1], bins=25, alpha=0.7, label='Treated (NSW)', 
        density=True, color=COLORS['observational'], edgecolor='white', linewidth=0.5)
ax.axvline(0.1, color=COLORS['neutral'], linestyle='--', linewidth=1, alpha=0.8, label='Trimming bounds')
ax.axvline(0.9, color=COLORS['neutral'], linestyle='--', linewidth=1, alpha=0.8)
ax.set_xlabel(r'Propensity Score $\hat{e}(X)$')
ax.set_ylabel('Density')
ax.set_title('(b) Observational Sample', fontweight='bold')
ax.legend(loc='upper right', framealpha=0.9)
ax.set_xlim([-0.02, 1.02])
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('../../results/figure1_propensity_overlap.pdf', bbox_inches='tight', dpi=300)
plt.savefig('../../results/figure1_propensity_overlap.png', bbox_inches='tight', dpi=300)
plt.show()

print("Figure 1 saved: results/figure1_propensity_overlap.pdf")

**Interpretation**: 
- The experimental sample shows good overlap with propensity scores clustered around 0.5 (reflecting randomisation).
- The observational sample shows poor overlap with the control group concentrated near 0 and the treated group spread across the range.

---

## 3. DML Estimation with $\kappa_{\text{DML}}$ Diagnostic

We estimate the average treatment effect (ATE) of the NSW job training program using DML with three nuisance learners of varying flexibility:

| Learner | Description | Flexibility |
|---------|-------------|-------------|
| **OLS** | Ordinary Least Squares | Parametric (low) |
| **LASSO** | L1-regularised regression | Sparse/linear (moderate) |
| **RF** | Random Forest | Nonparametric (high) |

The key insight from the paper is that more flexible learners can *exacerbate* the conditioning problem in observational designs by more accurately predicting treatment status, leaving less residual variation $\hat{U}_i = D_i - \hat{m}(X_i)$.

In [ ]:
# Define learner configurations (simple keys used to select default learners)

learner_configs = {

    'LIN': {},

    'LASSO': {},

    'RF': {}

}



# Get outcome

Y_exp = df_exp['re78'].values

Y_obs = df_obs['re78'].values



# DML settings

n_folds = 5

n_rep = 1  # Set to 10+ for publication



print(f"Running DML with K={n_folds} folds, {n_rep} repetition(s)")


In [ ]:
# Run DML for all designs and learners

results_list = []

for design_name, (X, D, Y) in [('Experimental', (X_exp, D_exp, Y_exp)), 
                                ('Observational', (X_obs, D_obs, Y_obs))]:
    print(f"\n{'='*50}")
    print(f"Design: {design_name}")
    print(f"{'='*50}")
    
    for learner_name, config in learner_configs.items():
        print(f"\n  Learner: {learner_name}")
        
        # Get learners
        ml_l = get_learner(learner_name)  # For E[Y|X]
        ml_m = get_learner(learner_name)  # For E[D|X]
        
        # Create and fit DML model
        dml = PLRDoubleMLKappa(
            learner_m=ml_m,
            learner_g=ml_l,
            n_folds=n_folds,
            random_state=42,
        )
        
        dml.fit(Y, D, X)
        
        # Extract results
        res = dml.summary_dict()
        # Map to expected keys used in notebook
        res['theta'] = res.pop('theta_hat')
        res['se'] = res.pop('se_dml')
        res['ci_lower'] = res.pop('ci_95_lower')
        res['ci_upper'] = res.pop('ci_95_upper')
        res['ci_length'] = res.pop('ci_length')
        res['kappa_dml'] = res['kappa_dml']
        res['n'] = res['n']
        
        res['Design'] = design_name
        res['Learner'] = learner_name
        results_list.append(res)
        
        print(f"    θ̂ = {res['theta']:.2f} (SE = {res['se']:.2f})")
        print(f"    κ_DML = {res['kappa_dml']:.2f}")
        print(f"    95% CI: [{res['ci_lower']:.2f}, {res['ci_upper']:.2f}]")

print("\nDML estimation complete.")

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results_list)

# Reorder columns
col_order = ['Design', 'Learner', 'n', 'theta', 'se', 'ci_lower', 'ci_upper', 
             'kappa_dml', 't_stat', 'p_value']
results_df = results_df[[c for c in col_order if c in results_df.columns]]

display(results_df.style.format({
    'theta': '{:.2f}',
    'se': '{:.2f}',
    'ci_lower': '{:.2f}',
    'ci_upper': '{:.2f}',
    'kappa_dml': '{:.2f}',
    't_stat': '{:.2f}',
    'p_value': '{:.4f}'
}))

In [ ]:
# Save results
results_df.to_csv('../../results/empirical_dml_results.csv', index=False)
print("Results saved to results/empirical_dml_results.csv")

---

## 4. The $\kappa_{\text{DML}}$ Diagnostic: Visualisation

The condition number $\kappa_{\text{DML}}$ directly measures the flatness of the orthogonal score. From the paper's refined linearization:

$$\hat{\theta} - \theta_0 = \kappa_{\text{DML}} \cdot (S_n + B_n) + R_n$$

where $S_n$ is the score average and $B_n$ is the nuisance estimation bias. This implies:

1. **Estimation error** scales as $\kappa_{\text{DML}}/\sqrt{n}$ (variance) plus $\kappa_{\text{DML}} \cdot r_n$ (bias)
2. **CI length** scales as $\kappa_{\text{DML}}/\sqrt{n}$
3. **Three regimes**: Well-conditioned ($\kappa \approx 1$), moderately ill-conditioned ($\kappa = O(n^\beta)$), severely ill-conditioned ($\kappa \asymp \sqrt{n}$)

In [ ]:
# =============================================================================
# Figure 2: Condition Number by Design and Learner (Publication Quality)
# =============================================================================

fig, ax = plt.subplots(figsize=(5.5, 4))

# Pivot for grouped bar chart
pivot_kappa = results_df.pivot(index='Learner', columns='Design', values='kappa_dml')
pivot_kappa = pivot_kappa.reindex(['LIN', 'LASSO', 'RF'])  # Ensure order
pivot_kappa = pivot_kappa[['Experimental', 'Observational']]  # Order columns

x = np.arange(len(pivot_kappa.index))
width = 0.35

bars1 = ax.bar(x - width/2, pivot_kappa['Experimental'], width, 
               label='Experimental', color=COLORS['experimental'], 
               edgecolor='white', linewidth=0.8, alpha=0.9)
bars2 = ax.bar(x + width/2, pivot_kappa['Observational'], width, 
               label='Observational', color=COLORS['observational'], 
               edgecolor='white', linewidth=0.8, alpha=0.9)

# Add threshold line
ax.axhline(y=10, color=COLORS['threshold'], linestyle='--', linewidth=1.5, 
           label=r'Threshold ($\kappa = 10$)', zorder=1)

ax.set_xlabel('Nuisance Learner')
ax.set_ylabel(r'Condition Number $\kappa_{\mathrm{DML}}$')
ax.set_title('DML Condition Number by Design and Learner', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(['OLS', 'LASSO', 'Random Forest'])
ax.legend(loc='upper left', framealpha=0.95)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8, fontweight='bold')

# Set y-axis to start at 0
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('../../results/figure2_kappa_by_design.pdf', bbox_inches='tight', dpi=300)
plt.savefig('../../results/figure2_kappa_by_design.png', bbox_inches='tight', dpi=300)
plt.show()

print("Figure 2 saved: results/figure2_kappa_by_design.pdf")

In [ ]:
# =============================================================================
# Figure 3: CI Length vs. Condition Number (Publication Quality)
# =============================================================================

fig, ax = plt.subplots(figsize=(5.5, 4.5))

results_df['ci_length'] = results_df['ci_upper'] - results_df['ci_lower']

# Scatter plot with different markers for design
markers = {'Experimental': 'o', 'Observational': 's'}
learner_colors = {'LIN': COLORS['LIN'], 'LASSO': COLORS['LASSO'], 'RF': COLORS['RF']}

for _, row in results_df.iterrows():
    ax.scatter(row['kappa_dml'], row['ci_length'], 
               marker=markers[row['Design']], 
               c=learner_colors[row['Learner']], 
               s=120, alpha=0.9, edgecolors='black', linewidth=0.8, zorder=5)

# Add reference line (theoretical relationship: CI length ∝ κ^{1/2})
kappa_range = np.linspace(0.5, results_df['kappa_dml'].max() * 1.2, 100)
# Baseline from well-conditioned experimental estimates
exp_mask = results_df['Design'] == 'Experimental'
baseline = results_df[exp_mask]['ci_length'].mean()
baseline_kappa = results_df[exp_mask]['kappa_dml'].mean()
ax.plot(kappa_range, baseline * np.sqrt(kappa_range / baseline_kappa), 
        color=COLORS['neutral'], linestyle='--', linewidth=1.5, alpha=0.7,
        label=r'Theoretical: $\propto \sqrt{\kappa}$', zorder=1)

# Threshold line
ax.axvline(x=10, color=COLORS['threshold'], linestyle=':', linewidth=1.5, 
           alpha=0.8, label=r'Threshold ($\kappa = 10$)', zorder=2)

ax.set_xlabel(r'Condition Number $\kappa_{\mathrm{DML}}$')
ax.set_ylabel('95% CI Length (\\$)')
ax.set_title('Confidence Interval Length vs. Condition Number', fontweight='bold')

# Custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=COLORS['neutral'], 
           markersize=8, markeredgecolor='black', markeredgewidth=0.5, label='Experimental'),
    Line2D([0], [0], marker='s', color='w', markerfacecolor=COLORS['neutral'], 
           markersize=8, markeredgecolor='black', markeredgewidth=0.5, label='Observational'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=COLORS['LIN'], 
           markersize=8, markeredgecolor='black', markeredgewidth=0.5, label='OLS'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=COLORS['LASSO'], 
           markersize=8, markeredgecolor='black', markeredgewidth=0.5, label='LASSO'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=COLORS['RF'], 
           markersize=8, markeredgecolor='black', markeredgewidth=0.5, label='RF'),
    Line2D([0], [0], linestyle='--', color=COLORS['neutral'], linewidth=1.5, 
           label=r'$\propto \sqrt{\kappa}$'),
]
ax.legend(handles=legend_elements, loc='upper left', framealpha=0.95, ncol=2)

ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('../../results/figure3_ci_length_vs_kappa.pdf', bbox_inches='tight', dpi=300)
plt.savefig('../../results/figure3_ci_length_vs_kappa.png', bbox_inches='tight', dpi=300)
plt.show()

print("Figure 3 saved: results/figure3_ci_length_vs_kappa.pdf")

**Key findings**:

1. **Experimental design**: $\kappa_{\text{DML}} \approx 4$ across all learners, reflecting the near-ideal overlap from randomisation.

2. **Observational design**: $\kappa_{\text{DML}}$ is substantially larger (especially with flexible learners like RF), indicating poor overlap and inflated standard errors.

3. **Learner effects**: More flexible learners (RF) tend to produce higher $\kappa_{\text{DML}}$ in the observational sample because they can more accurately predict treatment assignment, leaving less residual variation.

---

## 5. Robustness Analysis: Propensity Score Trimming

A natural response to poor overlap is to **trim** observations with extreme propensity scores. This trades off sample size for improved conditioning:

- **Benefits**: Reduces $\kappa_{\text{DML}}$ by removing observations where treatment is near-deterministic
- **Costs**: Reduces sample size and may change the estimand (ATT on a restricted population)

We examine symmetric trimming thresholds $\alpha \in \{0, 0.01, 0.05, 0.10, 0.15\}$, dropping observations with $\hat{e}(X) < \alpha$ or $\hat{e}(X) > 1 - \alpha$.

In [ ]:
# Trimming analysis for observational sample

trim_thresholds = [0.0, 0.01, 0.05, 0.10, 0.15]

trim_results = []



for trim in trim_thresholds:

    # Apply symmetric trimming

    mask = (ps_obs > trim) & (ps_obs < 1 - trim)

    n_trimmed = len(ps_obs) - mask.sum()

    

    if mask.sum() < 50:  # Skip if too few observations

        continue

    

    X_trim = X_obs[mask]

    D_trim = D_obs[mask]

    Y_trim = Y_obs[mask]

    

    # Run DML with linear learner

    ml_l = get_learner('LIN')

    ml_m = get_learner('LIN')

    

    dml = PLRDoubleMLKappa(learner_m=ml_m, learner_g=ml_l, n_folds=5, random_state=42)

    dml.fit(Y_trim, D_trim, X_trim)

    

    res = dml.summary_dict()

    # Map to expected keys

    res['theta'] = res.pop('theta_hat')

    res['se'] = res.pop('se_dml')

    res['ci_lower'] = res.pop('ci_95_lower')

    res['ci_upper'] = res.pop('ci_95_upper')

    res['ci_length'] = res.pop('ci_length')

    res['kappa_dml'] = res['kappa_dml']

    res['n'] = res['n']

    

    res['trim_threshold'] = trim

    res['n_trimmed'] = n_trimmed

    res['pct_trimmed'] = n_trimmed / len(ps_obs) * 100

    trim_results.append(res)



trim_df = pd.DataFrame(trim_results)

display(trim_df[['trim_threshold', 'n', 'n_trimmed', 'pct_trimmed', 'theta', 'se', 'kappa_dml']].style.format({

    'pct_trimmed': '{:.1f}%',

    'theta': '{:.2f}',

    'se': '{:.2f}',

    'kappa_dml': '{:.2f}'

}))


In [ ]:
# =============================================================================
# Figure 4: Trimming Analysis (Publication Quality)
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(7, 3.5), layout='constrained')

# Panel (a): κ_DML vs trimming threshold
ax = axes[0]
ax.plot(trim_df['trim_threshold'] * 100, trim_df['kappa_dml'], 
        'o-', color=COLORS['experimental'], linewidth=2, markersize=8, 
        markeredgecolor='white', markeredgewidth=1)
ax.axhline(y=10, color=COLORS['threshold'], linestyle='--', linewidth=1.5, 
           alpha=0.8, label=r'Threshold ($\kappa = 10$)')
ax.set_xlabel('Trimming Threshold (%)')
ax.set_ylabel(r'Condition Number $\kappa_{\mathrm{DML}}$')
ax.set_title(r'(a) Effect of Trimming on $\kappa_{\mathrm{DML}}$', fontweight='bold')
ax.legend(loc='upper right', framealpha=0.95)
ax.set_xlim(left=-0.5)
ax.set_ylim(bottom=0)

# Panel (b): Sample size vs κ_DML trade-off
ax = axes[1]
scatter = ax.scatter(trim_df['n'], trim_df['kappa_dml'], 
                     c=trim_df['trim_threshold'] * 100, cmap='viridis', 
                     s=150, edgecolors='black', linewidth=1, zorder=5)
for _, row in trim_df.iterrows():
    ax.annotate(f"{row['trim_threshold']*100:.0f}%", 
                xy=(row['n'], row['kappa_dml']),
                xytext=(8, 3), textcoords='offset points', fontsize=8)
ax.axhline(y=10, color=COLORS['threshold'], linestyle='--', linewidth=1.5, alpha=0.8)
ax.set_xlabel('Sample Size $n$')
ax.set_ylabel(r'Condition Number $\kappa_{\mathrm{DML}}$')
ax.set_title('(b) Sample Size vs. Conditioning Trade-off', fontweight='bold')

# Add colorbar
cbar = fig.colorbar(scatter, ax=ax, shrink=0.8, pad=0.02)
cbar.set_label('Trim (%)', fontsize=9)

ax.set_ylim(bottom=0)

plt.savefig('../../results/figure4_trimming_analysis.pdf', bbox_inches='tight', dpi=300)
plt.savefig('../../results/figure4_trimming_analysis.png', bbox_inches='tight', dpi=300)
plt.show()

print("Figure 4 saved: results/figure4_trimming_analysis.pdf")

### 5.2 Comparison Across All Learners

In [ ]:
# =============================================================================
# Figure 5: Forest Plot of Treatment Effect Estimates (Publication Quality)
# =============================================================================

fig, ax = plt.subplots(figsize=(6.5, 4.5))

# Sort by Design then Learner
results_df_sorted = results_df.sort_values(['Design', 'Learner'], ascending=[False, True])
y_pos = np.arange(len(results_df_sorted))

# Create labels
labels = [f"{row['Design']} — {row['Learner']}" for _, row in results_df_sorted.iterrows()]

# Plot each point individually with appropriate colors
for i, row in enumerate(results_df_sorted.itertuples(index=False)):
    if 'Experimental' in labels[i]:
        point_color = COLORS['experimental']
    else:
        point_color = COLORS['observational']
    
    ax.errorbar(row.theta, y_pos[i],
                xerr=1.96 * row.se,
                fmt='o', color=point_color, ecolor=point_color, 
                elinewidth=2, capsize=4, capthick=1.5, markersize=8,
                markeredgecolor='white', markeredgewidth=0.8, zorder=5)

# Reference line at 0
ax.axvline(x=0, color=COLORS['neutral'], linestyle='-', linewidth=0.8, alpha=0.5, zorder=1)

# Experimental benchmark (from LaLonde 1986 / Dehejia-Wahba 1999)
EXPERIMENTAL_BENCHMARK = 1794
ax.axvline(x=EXPERIMENTAL_BENCHMARK, color=COLORS['benchmark'], linestyle='-', 
           linewidth=2, alpha=0.7, label=f'Experimental benchmark (\\${EXPERIMENTAL_BENCHMARK:,})', zorder=2)

# Add shaded region for benchmark uncertainty (illustrative)
ax.axvspan(EXPERIMENTAL_BENCHMARK - 500, EXPERIMENTAL_BENCHMARK + 500, 
           alpha=0.1, color=COLORS['benchmark'], zorder=0)

ax.set_yticks(y_pos)
ax.set_yticklabels(labels)
ax.set_xlabel('Treatment Effect on Earnings (\\$)')
ax.set_title('DML Estimates with 95\\% Confidence Intervals', fontweight='bold')
ax.legend(loc='lower right', framealpha=0.95)

# Add grid only on x-axis
ax.xaxis.grid(True, alpha=0.3)
ax.yaxis.grid(False)

# Adjust spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('../../results/figure5_forest_plot.pdf', bbox_inches='tight', dpi=300)
plt.savefig('../../results/figure5_forest_plot.png', bbox_inches='tight', dpi=300)
plt.show()

print("Figure 5 saved: results/figure5_forest_plot.pdf")

---

## 6. Summary and Conclusions

### Key Findings

This empirical application demonstrates the practical relevance of the DML condition number diagnostic using the canonical LaLonde job training data:

1. **Experimental vs. Observational**: The condition number $\kappa_{\text{DML}}$ is **substantially higher** in the observational sample (PSID comparison group) than in the experimental sample, reflecting the poor overlap between NSW participants and non-experimental controls.

2. **Learner Effects**: More flexible nuisance learners (Random Forest) can **exacerbate** the conditioning problem by more accurately predicting treatment assignment. This leaves less residual treatment variation, inflating $\kappa_{\text{DML}}$.

3. **CI Length Scaling**: Confidence interval length scales approximately as $\sqrt{\kappa_{\text{DML}}}$, consistent with the paper's theoretical predictions.

4. **Trimming Trade-offs**: Propensity score trimming improves conditioning but at the cost of sample size. There is a **bias-variance trade-off** in choosing the trimming threshold.

### Practical Guidance

When $\kappa_{\text{DML}} > 10$, practitioners should:
- ⚠️ Investigate propensity score overlap diagnostics
- ✂️ Consider trimming extreme propensity scores
- 📉 Use simpler nuisance learners if flexible ones produce high $\kappa$
- 📊 Report $\kappa_{\text{DML}}$ alongside DML estimates as a routine diagnostic

### Analogy to Instrumental Variables

Just as the first-stage $F$-statistic and minimum eigenvalue diagnostics flag weak instruments in IV regression, $\kappa_{\text{DML}}$ flags weak conditioning in DML. Both diagnostics help practitioners assess when standard asymptotic inference may fail in finite samples.

In [ ]:
# Final summary table
summary_table = results_df.pivot_table(
    index='Design',
    columns='Learner',
    values=['theta', 'se', 'kappa_dml'],
    aggfunc='first'
)

# Flatten column names
summary_table.columns = [f"{col[0]}_{col[1]}" for col in summary_table.columns]

print("=== Summary Table ===")
display(summary_table.round(2))

In [ ]:
# Export to LaTeX
latex_table = results_to_latex(
    results_df, 
    caption="DML estimates with condition number diagnostic",
    label="tab:empirical"
)

with open('../../results/empirical_results.tex', 'w') as f:
    f.write(latex_table)

print("LaTeX table saved to results/empirical_results.tex")
print("\n" + latex_table)

---

## Reproducibility Notes

### Session Information

In [ ]:
import sklearn
import scipy

print("=== Session Information ===")
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")
print(f"SciPy: {scipy.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")

In [ ]:
# List output files
import os

results_dir = '../../results'
print("\n=== Output Files ===")
for f in sorted(os.listdir(results_dir)):
    if f.startswith('empirical') or f.endswith('.pdf'):
        path = os.path.join(results_dir, f)
        size = os.path.getsize(path)
        print(f"  {f}: {size:,} bytes")

---

## Appendix: Data Sources and Replication

### Data Sources

| Dataset | Description | Reference |
|---------|-------------|-----------|
| **NSW Experimental** | Randomised job training experiment | LaLonde (1986), *AER* |
| **PSID Comparison** | Non-experimental control group | Dehejia & Wahba (1999), *JASA* |

### References

- LaLonde, R.J. (1986). "Evaluating the Econometric Evaluations of Training Programs with Experimental Data." *American Economic Review*, 76(4), 604–620.

- Dehejia, R.H. and Wahba, S. (1999). "Causal Effects in Nonexperimental Studies: Reevaluating the Evaluation of Training Programs." *Journal of the American Statistical Association*, 94(448), 1053–1062.

- Chernozhukov, V., Chetverikov, D., Demirer, M., Duflo, E., Hansen, C., Newey, W., and Robins, J. (2018). "Double/Debiased Machine Learning for Treatment and Structural Parameters." *The Econometrics Journal*, 21(1), C1–C68.

### Replication

```bash
# Clone repository and install dependencies
git clone https://github.com/gsaco/dml_paper.git
cd dml_paper/src/empirical
pip install -r requirements.txt

# Run this notebook
jupyter notebook empirical_application.ipynb
```

### Output Files

All figures are saved to `results/` in both PDF (for LaTeX) and PNG (for preview) formats.